In [4]:
import os

In [5]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\Projects\\13.wine_quality_prediction_project\\research'

In [6]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\Projects\\13.wine_quality_prediction_project'

In [ ]:
#1.config.yaml updated
#2.params updated as per model parameter
#3.entity updated
#4.configurationManager updated from src
#5.componet updated
#6.pipeline updat

In [13]:
from dataclasses import dataclass
from pathlib import Path
import joblib

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
                                                                                                                                                


In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        return model_trainer_config

In [16]:
import os
from sklearn.linear_model import ElasticNet
from pathlib import Path
import pandas as pd
import joblib

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train_model(self):
        train = pd.read_csv(self.config.train_data_path)
        test = pd.read_csv(self.config.test_data_path)

        #now here we have to get X_train, y_train and X_test,y_test
        X_train = train.drop([self.config.target_column],axis=1)
        X_test = test.drop([self.config.target_column],axis=1)

        y_train = train[[self.config.target_column]]
        y_test = test[[self.config.target_column]]

        model =ElasticNet(l1_ratio=self.config.l1_ratio, alpha=self.config.alpha,random_state=42)
        model.fit(X_train,y_train)

        #saving model as pkl file with joblib lib
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))


In [17]:
try:
    config = ConfigurationManager()
    model_trainer_config =config.get_model_trainer_config()
    start_model_trainer =ModelTrainer(config = model_trainer_config)
    start_model_trainer.train_model()
except Exception as e:
    raise e

[2024-07-05 17:19:57,674: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-07-05 17:19:57,674: INFO:common:yaml file: params.yaml loaded successfully]
[2024-07-05 17:19:57,683: INFO:common:yaml file: schema.yaml loaded successfully]
[2024-07-05 17:19:57,684: INFO:common:created directory at: artifacts]
[2024-07-05 17:19:57,684: INFO:common:created directory at: artifacts/model_trainer]
